# 3. Working with data

## 3.1 Data format

* spreadsheet
* pandas library
* scrape from webpage
* read from file

## 3.2 Manipulating data

* pandas indexing
* new columns etc.
* statistics
* histograms, plots with error bars

## 3.4 More plotting

* facets